<a href="https://colab.research.google.com/github/dablro12/AI_study/blob/main/OS_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 제출 마감일 : 5월 21일 (일요일) 10:59pm
늦으면 eclass 가 막혀서 제출이 불가, eClass 과제방에 제출
* 제출 양식
설치 과정을 간단히 설명하는 문서
실행 가능한 소스코드도 제출
* 사용 언어
Python

# 과제3 - 교차로

간단한 세마포의 예로 교차로 문제를 다루었다.
여러 개의 기차가 있지만 한 번에 이용할 수 있는 기차는 1개 뿐이다.
교차로 이용권을 세마포(또는 뮤텍스락)로 만들어 기차들이 교차로에서 충돌하지 않도록 하였다.
기차가 교차로에 들어간 시점과 나오는 시점이 표시되어 한 번에 하나의 기차만 교차로를 통과하는 것을 알 수 있었다.
- 이번에는 좀 더 복잡한 도로의 교차로 상황에서 차들의 통행을 동기화하는 문제이다.
- 십자 형태로 교차하는 교차로의 동서남북 4개의 방향에서 차들이 진입한다.
- 이 차들이 충돌하지 않도록 조율하여 모든 차들이 잘 지나갈 수 있도록
하는 것이 우리의 목표이다.


- 위아래에서 차가 들어오고 차들은 좌회전, 직진 할 수 있다.
- 직진하는 차량은 교차로 내의 블록 중에 2개를, 좌회전하는 차량은 교차로 내의 블록 중에 3개를 사용한다.
- 직진하는 차량은 블록 2개를 쓸 수 있어야 직진이 가능하고, 좌회전하는 차량은 블록 3개를 쓸 수 있어야 좌회전이 가능하다.

# 우리가 해야 할 일(1) : 교착상태 만들기
- 차는 모두 12대가 준비되어 있다. 위에서 내려오는 차 3대는 좌회전, 3대는 직진할 것이다. 아래에서 올라가는 차도 3대는 좌회전, 3대는 직진할 것이다.
- 위에서 좌회전하는 쓰레드 3개, 직진하는 쓰레드 3개, 아래에서 좌회전하는쓰레드 3개, 직진하는 쓰레드 3개를 만든다. 이 12개의 쓰레드를 동시에 동작시킨다. 4 종류의 쓰레드를 만드는 것은 4개의 함수를 만드는 것이다.
- 각 쓰레드는 진입을 위해 교차로의 블록을 예약해야 한다. 교차로의 블록을 예약하면 차(=쓰레드)가 진입한다. 쓰레드 함수 안에 블록을 예약하는 동기화 도구를 써야 한다.
- 의도적으로 교착 상태를 만들자. 동기화 도구를 조절하여 차들이 교착상태에 빠지게 만들자. 교착 상태에 빠지면 아무 것도 하지 못한 채 차들이 서 있을것이다. 프로그램은 종료되지 못하고 모두 서 있게 된다.
* 왜 교착 상태에 빠졌고, 그 때의 코드는 무엇이고, 결과 화면은 어떤 것인지 캡처하여 보고서에 넣는다. 교착 상태에 빠진 것은 어떻게 판단할까?

* 1~3번기차 :위에서 직진 / 4~6번 기차 : 위에서 좌회전 / 7~9번 기차 : 아래에서 직진 / 10~12번 기차: 아래에서 좌회전


In [ ]:
#thread 리스트로 맞춰주고     
def make_threads(num):
  dir_threads = []
  for i in range(num):
    ### 각 쓰레드는 진입을 위해 교차로의 블록을 예약해야 한다. 교차로의 블록을
    ### 예약하면 차(=쓰레드)가 진입한다. 쓰레드 함수 안에 블록을 예약하는 동기화 도구를 써야 한다    
    if i <3:
      dir_threads.append(threading.Thread(target = run, args = (i, 'NS', 2)))
    elif i >= 3 and i < 6:
      dir_threads.append(threading.Thread(target = run, args = (i, 'NL', 2)))
    elif i >= 3 and i < 6:
      dir_threads.append(threading.Thread(target = run, args = (i, 'SS', 3)))
    else:
      dir_threads.append(threading.Thread(target = run, args = (i, 'SL', 3)))
  return dir_threads  

In [ ]:
#thread module 가지고오기 
import threading, time 
#세마포어 교차로에는 최대 2개까지만 들어갈 수 있음
sem = threading.Semaphore(2)

def run(n, args):
  t = 3
  sem.acquire(blocking= True)
  status = [0,0] #위쪽, 아래쪽에는 한개씩만 존재해야함

  if args in ['NS', 'SS']:
    print(f"세마포 status : {sem._value}")
  
    for x in range(2):
      if x == 1: #두번쨰 블록에서 직진,
        print(f'{args} 방향의 {n+1}번 기차가 두번쨰 블록을 직진합니다. 상태 : {x} block\t')
        print(f'{args} 방향의 {n+1}번 기차 상태 : {x+1} block')
        #print(f"세마포 status : {sem._value}\n")
        
        time.sleep(t)
      else: #첫번쨰 블록에서 직진
        print(f'{args} 방향의 {n+1}번 기차가 첫번째 블록으로 직진하기 시작합니다. {x} block\t')
        print(f'{args} 방향의 {n+1}번 기차 상태 : {x+1} block')
        #print(f"세마포 status : {sem._value}\n")
        status
        
        time.sleep(t)
    if x == 1: #두번쨰 블록으로 왔으면 메시지 말해줌
      #print(f"세마포 status : {sem._value}")
      print(f'{args} 방향의 {n+1}번 기차가 교차로를 통과했습니다.\n')

  elif args in ['NL', 'SL']:
    print(f"세마포 status : {sem._value}")
    
    for x in range(3):
      if x == 2:#세번쨰 블록에서는 는 직진
        print(f'{args} 방향의 {n+1}번 기차가 세번쨰 블록을 직진합니다. {x} block\t')
        print(f'{args} 방향의 {n+1}번 기차 상태 : {x+1} block\t')
        #print(f"세마포 status : {sem._value}\n")
        
        time.sleep(t)
      elif x == 1:#두번쨰 블록에서는 좌회전
        print(f'{args} 방향의 {n+1}번 기차가 두번쨰 블록에서 좌회전합니다. {x} block\t')
        print(f'{args} 방향의 {n+1}번 기차 상태 : {x+1} block\t')
        #print(f"세마포 status : {sem._value}\n")
        
        time.sleep(t)
      else:#첫번쨰 블록에서는 무조건 직진
        print(f'{args} 방향의 {n+1}번 기차가 첫번째 블록으로 직진하기 시작합니다. {x} block\t')
        print(f'{args} 방향의 {n+1}번 기차 상태 : {x+1} block\t')
        #print(f"세마포 status : {sem._value}\n")
        
        time.sleep(t)
    if x == 2:
      #print(f"세마포 status : {sem._value}")
      print(f'{args} 방향의 {n+1}번 기차가 교차로를 통과했습니다.\n')

  sem.release()
  



